In [256]:
import pandas as pd
import functools
import numpy as np

In [321]:
x

,First Name,Last Name,Display Name,Nickname,E-mail Address,E-mail 2 Address,E-mail 3 Address,Home Phone,Business Phone,Home Fax,...,Notes,Birthday,Anniversary,Gender,Web Page,Web Page 2,Categories,Unnamed: 37,Unnamed: 38,Unnamed: 39
4052,Beverly,Melugin,Beverly Melugin,NaN,Beverly.Melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4053,Beverly,Melugin,Beverly Melugin,NaN,beverly.melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4054,Beverly,Melugin,Beverly Melugin,NaN,beverly.melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4055,Beverly,Melugin,Beverly Melugin,NaN,Beverly.Melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4056,Beverly,Melugin,Beverly Melugin,NaN,Beverly.Melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4057,Beverly,Melugin,Beverly Melugin,NaN,beverly.melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [344]:
c = pd.read_csv("contacts.csv")

v = c.loc[~c["First Name"].isnull() & c["First Name"].str.contains("'"),["First Name","Last Name"]]
v = v.values.tolist()
def splitname(name):
    n = name.split(" ")
    if len(n)==1:
        return [n[0],None]
    return [" ".join(n[:-1]), n[-1]]
newNames = [splitname(first.strip("'")) if first[0]=="'" and first[-1]=="'" else [first,last] for first,last in v ]
newNames


[['Allison', 'Bloom'],
 ['Allison', 'Bloom'],
 ['Allison', 'Bloom'],
 ['Allison', 'Bloom'],
 ['Allison', 'Bloom'],
 ["St. Mary's Quick Care", 'Adult clinic'],
 ["St. Mary's Quick Care", 'Adult clinic'],
 ['Barnum', None],
 ['Barnum', None],
 ["Lotta's", 'Bakery'],
 ["Lotta's", 'Bakery'],
 ['Camilla', 'Newhagen'],
 ['Camilla', 'Newhagen'],
 ['Claire', 'Pollioni'],
 ['Claire', 'Pollioni'],
 ["Child's delight", nan],
 ["Jonathan's", 'Chinese Restaurant'],
 ["Jonathan's", 'Chinese Restaurant'],
 ['Dante and Mary', 'Menicucci'],
 ['Dante and Mary', 'Menicucci'],
 ['Davd Koff', '(davidkoff@gmail.com)'],
 ['Davd Koff', '(davidkoff@gmail.com)'],
 ['Davd Koff', '(davidkoff@gmail.com)'],
 ['Davd Koff', '(davidkoff@gmail.com)'],
 ["'Dgteam", nan],
 ["'Dgteam", nan],
 ["Miller's East Coast", 'Deli'],
 ["Miller's East Coast", 'Deli'],
 ['Erin', 'Shallat'],
 ['Erin', 'Shallat'],
 ["Eliza's", nan],
 ["Eliza's", nan],
 ["Giorgio's Pizza", nan],
 ["Giorgio's Pizza", nan],
 ['Michael', None],
 ['Michael

In [378]:
class Contacts:
    def __init__(self):
        self.Contacts = pd.read_csv("contacts.csv")
        self.fixNames()
        b = len(self.Contacts)
        self.Contacts = self.Contacts.drop_duplicates(ignore_index=True)
        dup = b - len(self.Contacts) 
        #self.add()
        a = len(self.Contacts)
        print("before:",b, "after:",a, "# of duplicates:",dup, "# compressed:", b-a)
        i = self.Contacts.groupby(["First Name","Last Name"])["Home Phone"].count()
        print("# of different people (unique first and last names):", len(i))
        self.Contacts["Display Name"] = self.Contacts[["First Name","Last Name"]].apply(self.replaceDisplayName,axis=1)
    
    def replaceDisplayName(self,x):
        f = x["First Name"]
        l = x["Last Name"]
        if f!=f:
            return l
        if l!=l:
            return f
        print(l,f,l!=l)
        return f + " " + l
    
    def splitname(self,name):
        n = name.split(" ")
        if len(n)==1:
            return [n[0],None]
        return [" ".join(n[:-1]), n[-1]]
    def fixNames(self):
        v = self.Contacts.loc[~self.Contacts["First Name"].isnull() & self.Contacts["First Name"].str.contains("'"),["First Name","Last Name"]].values.tolist()
        newNames = [self.splitname(first.strip("'")) if first[0]=="'" and first[-1]=="'" else [first,last] for first,last in v ]
        self.Contacts.loc[~self.Contacts["First Name"].isnull() & self.Contacts["First Name"].str.contains("'"),["First Name","Last Name"]] = newNames

    
    def merge(self,person):
        newperson = []
        for feature in person.columns:
            vals = list(person[feature].dropna().unique())
            if len(vals)==0:
                vals = [None]
            newF = functools.reduce(lambda a, b: str(a)+" "+str(b), vals)
            newperson.append(newF)
        return newperson
    def createNewPersonContacts(self, person):
        less_than_one_phone = len(person["Home Phone"].dropna().unique())<=1
        less_than_one_email = len(person["E-mail Address"].dropna().unique())<=1
        if less_than_one_phone and less_than_one_email:
            newperson = [self.merge(person)]
        elif not less_than_one_email and less_than_one_phone:
            newperson = person.groupby("E-mail Address").apply(self.merge).values
        elif not less_than_one_phone and less_than_one_email: 
            newperson = person.groupby("Home Phone").apply(self.merge).values
        else:
            newperson = person.groupby("E-mail Address").apply(self.merge).values
        newperson = list(newperson)
        return newperson
                
                
    def add(self):
        c = self.Contacts.groupby(["First Name","Last Name"],dropna=False)
        newC = []
        o = 0
        for i,person in c:
            newperson = self.createNewPersonContacts(person) 
            o += len(person)
            newC += newperson
        
        print(len(self.Contacts),o)
            
        self.Contacts = pd.DataFrame(newC,columns=self.Contacts.columns)
        self.Contacts = self.Contacts.sort_values(by=["First Name","Last Name"])

In [379]:
c = Contacts()
c.Contacts.to_csv("newContacts.csv")

before: 6771 after: 4148 # of duplicates: 2623 # compressed: 2623
# of different people (unique first and last names): 1956
Bloom Allison False
Bloom Allison False
(Aniko Skyler) Sye Perry False
AA Sall Jacobs False
Abendano Elizabeth False
Abruzzese Martha False
Accountant Tina Hall False
Acuri Meeta False
Adams Elizabeth False
Adams Jon False
Adams Lyncuoya False
Adler H False
Adshead Laura False
Adult clinic St. Mary's Quick Care False
Adult clinic St. Mary's Quick Care False
Africa David False
Ahmed Rachel False
Ai Dr. False
Ai Dr. King False
AI Everett False
Airport/train Shuttle Rome False
Alahendra Pri False
Alarcio Faith False
Alavi Kian False
Alexander David False
Alexander Michael & Liane False
Alexander Simon False
Alexander Simon & Agnes False
Allan Peter False
Allen Peter False
AllState 37292921 False
Alpers Nora False
Alpert Gary False
Alterman Wallack Rachel False
Alvarado Franklin False
Alvarez Ashley False
American Express Kristofer Funkhouser False
American Express Kr

TypeError: can only concatenate str (not "NoneType") to str

In [371]:
def i(x):
    print(x["First Name"])
    return x
c.Contacts["Display Name"] = c.Contacts[["First Name"]].apply(i,axis=1)

Allison
Allison
Perry
A Johnson
A Johnson
A Johnson
A16
a2zbrenner@sbcglobal.net
Sall Jacobs
nan
Aaron Noveshen
Aaron Pollock
Elizabeth
abills@bulldogsolutions.com
Martha
nan
Tina Hall
Ace plumbing
Meeta
nan
nan
Elizabeth
Jon
Lyncuoya
adcflyer@hotmail.com
Adira Shalev
H
nan
Laura
St. Mary's Quick Care
St. Mary's Quick Care
nan
Afonso
David
nan
ageist@yahoo.com
Rachel
Dr.
Dr. King
Everett
nan
nan
nan
Rome
nan
a.karr@orbisfunds.com
a.karr@orbisfunds.com
a.karr@orbisfunds.com
al@fortmason.org
Pri
nan
Alan
Alan
Alan Winters
Faith
Kian
Albert
Alden
Alden
alebarbara@hotmail.com
nan
Alex
David
Michael & Liane
Simon
Simon & Agnes
Alexander Stack
Alexandra
alexis.limberakis@clorox.com
nan
alicia.neiman@clorox.com
ali.guermazi@synaro.com
ali_guermazi@synaro.com
alisa brozinsky
Alisa Yuko Brozinsky
Alison Wong
Peter
Allan
Peter
nan
Allison Healy Poe
allison.lai.wg96@wharton.upenn.edu
nan
37292921
Alpers, Ann
Nora
Gary
nan
nan
Rachel
Franklin
Ashley
Alyss.Dixson@clorox.com
nan
Amber Park
Amelia
Am

Michael & Valerie
Valerie
Valerie
Valerie
Valerie
harv
Mona
Ken
Lisa Emma Cole
Lisa  Emma Cole
Lisa
John
Lizzy
Cathy& Jon & Madeline& AJ
Cathy& Jon & Madeline& AJ
Sandra
Britt
Nicholas
Laure
Brendan
Meg
Tracey
Tracey
Tracey
Hedy and Joel Rutman
Hedy and Joel Rutman
Heidi
heididbug@comcast.net
Anna
Helen Ip-Lee
heleny@earthlink.net
nan
Hemanth.Reddy@clorox.com
Barbara
Henrike
Lisa
Lisa
Lisa
Mahdi
Ben
Ben
nan
Chris
John
Kaye
Lindsey, John, Peter, Fletcher
Lindsey, John, Peter, Fletcher
Lyndsey
Kathleen
Hershey 2
Hershey 3
Charles
John
heyelena@comcast.net
HH
HHonors
Andrea
Andrea
Andrea
Gateway
nan
Highbury Tennis Courts
Lolita
Lynn
Tiffany
nan
Hima Pamoedjo
Dan
Michael
Michael & Nancy & Sophie
Nancy
Nancy & Mike
Dan
Cece
David
Remi
Rebecca
Max
Steven
Seema
Seema
Bill
Hiro.Agata.wg96@wharton.upenn.edu
Hiromitsu_Agata@iog-24.ccgw.nex.co.jp
Juliette
Juliette
Hitoshi Hirose
Cat
Cat
Liz
Liz & Xavier
Liz & Xavier
Liz & Xavier
Liz & Xavier
Carol
Carol
maria
nan
Rachel Dave
Holly
Evelyn
Elena &

Donna
Margarita
Debbie
Nelanie
Arun
Jessica
nan
nan
Kennedy@gte.
Michael Sutton
Michael Sutton
Tal
David Jamie Hayes
David Jamie Hayes
David Jamie Hayes
David Jamie Hayes
David Jamie Hayes
Jamie
Jamie
Jamie
Jamie
Jamie
Jamie and Dave
Bonnie
Bonnie
Bonnie
Mary Grace
New Leaf Florist
Babysitter
nan
Kate
Desmond
Nice cuts
Nicholas
Cale, Tom, Barrett, & Samantha, Wyatt  Nichols
Marianna
Paul
Nick
nan
nick.feimer@clorox.com
Maria
Nicole Elaine Janin Sheehan
Nicole.Pineda@clorox.com
Erin
Erin
Stacey
Nina
Nina & Doug Thompson
Coach Jose
nishime@sonoma.edu
nan
nan
Nob Hill Spa
nan
Jose
Dawn
Natalie
Niklas & Teresa
nan
Bob
nan
nan
nan
NORMAN MARSHALL
Jon
Leslie
Leslie
nan
Amit
nan
James
noura_guermazi@yahoo.com
nan
Aaron
nan
NPS
NPS Maintenance
NPS
nsgomes@sherithisrael.org
NShef@aol.com
Ulisses
Nurseline
Jeff
Jeff
nan
1800nytimes
Carol
Veronica
Rachel
rachel & marty
rachel & marty
Ocean Aquarium
Ocean Aquarium
Erin
Erin
nan
Chris
Kristina
Shannon
office@c5children.org
Office depot
Scott
Tatsur

Adult Clinic
nan
Stacey
Stacy Antia
Stacy Dorter
Shane
Stuart
Sara
nan
Amit
Jone
Stefanee.Bartolotta@clorox.com
Stefany glass
Bettina
Margot Lockwood
Tom
Tom & Margot
Ryan
Jill
Jill
Stephanie Chai
Stephanie Dawson
Stephanie Dawson
Stephanie.Dawson@gap.com
Stephen Elliot
Elena
Mike
stephshea@aol.com
Lauran Adam Eli Clara
Marjorie
Jeff
Debbie
Steve Friedman
Steve Friedman/bryce Nash
nan
Steven
Steven
Steven Hurwitz
Bob
Lisa
Brian
Brian Camilla Nilolas
Paula
Lee
Mike
Jill
Leah
Annie
Yvonne
hf
Sue Painter
Dr Robert
Robert
Sugar bowl Lodge
Sugar bowl Lodge
Sugarbowl Daycare
Sugarbowl Powdercubs
Sugarbowl Training Center
Suha Hourani
Embassy
Sutchitha
Dana
Dana
Amy Lipetz
Gee
Sunita.L.Ganapati@wellsfargo.com
Sunset Playground
nan
Supercuts
Emer
support@surveymonkey.com
Party Staff
Party Staff
Susan Claxton
Susan Drell
Gillett-Kumli
SUSAN JONES
Susan Mohun Sherman
Susan Pannullo
Susan Wittenberg & Josh Dorosin
Susan.Hartman.wg96@wharton.upenn.edu
susan_lipetz@mercer.com
susan.lipetz@ siemens.

In [369]:
c.Contacts

,First Name,Last Name,Display Name,Nickname,E-mail Address,E-mail 2 Address,E-mail 3 Address,Home Phone,Business Phone,Home Fax,...,Notes,Birthday,Anniversary,Gender,Web Page,Web Page 2,Categories,Unnamed: 37,Unnamed: 38,Unnamed: 39
0,Allison,Bloom,NaN,NaN,bloomfive@gmail.com,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allison,Bloom,NaN,NaN,bloomfive@gmail.com,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Perry,(Aniko Skyler) Sye,NaN,NaN,perrysye@gmail.com,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A Johnson,NaN,NaN,NaN,mmmgmt@gmail.com,NaN,NaN,NaN,NaN,NaN,...,Yuyyyyyyyyyyy is,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A Johnson,NaN,NaN,NaN,mmmgmt@gmail.com,NaN,NaN,NaN,NaN,NaN,...,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4143,Boba Tea,36th And Balboa Near Polo Fields,NaN,NaN,NaN,NaN,NaN,(415) 213-5779,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41541733772,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4145,NaN,42 Degree's d,NaN,NaN,NaN,NaN,NaN,4157775558,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4146,Stuart Kesselman Mr Ticket,49er Or Baseball Or Other Tix Tickets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [381]:
np.nan != np.nan

True